In [1]:
import geopandas as gpd
import pandas as pd
import os,re,json
import numpy as np
from p_tqdm import p_map

In [2]:

# pip install mbutil

In [3]:
# lets generate the overview pictures 
#
floc = "tmp/county.geojson"
# gpd.read_file('data/CountiesUA.geojson')[['OBJECTID', 'CTYUA20CD', 'CTYUA20NM','LONG', 'LAT', 'geometry']].to_file(floc, driver="GeoJSON")

gdf = gpd.read_file('data/CountiesUA.geojson')[['CTYUA20CD', 'geometry']]
gdf.columns = ['id','geometry']
gdf.to_file(floc, driver="GeoJSON")


os.system('tippecanoe --read-parallel -Q -P -Z0 -z6 -f -o tiles/countries.mbtiles'+ floc)


tippecanoe: tiles/countries.mbtilestmp/county.geojson: unable to open database file


256

In [4]:
#  gpd.read_file('data/centroids2001.geojson').columns
gdf.head()


,id,geometry
0,E06000001,"POLYGON ((-1.27025 54.72717, -1.26822 54.72609..."
1,E06000002,"MULTIPOLYGON (((-1.19752 54.58211, -1.19434 54..."
2,E06000003,"MULTIPOLYGON (((-1.20098 54.57763, -1.20030 54..."
3,E06000004,"MULTIPOLYGON (((-1.27113 54.55427, -1.27213 54..."
4,E06000005,"POLYGON ((-1.63800 54.61720, -1.63767 54.61670..."


In [11]:
year = '01'

base  = gpd.read_file('data/OA2001.geojson')[['OA01CD', 'geometry']]
base.columns = ['id','geometry']
centroids = gpd.read_file('data/centroids2001.geojson')[['OA01CD','geometry']]

print('save')
base.to_file("tmp/OA%s.geojson"%year, driver="GeoJSON")
centroids.to_file("tmp/centroids%s.geojson"%year, driver="GeoJSON")

print('process')
start = 6
end = 12
# cant have centroids being grouped, but fine for area shapes
os.system('tippecanoe --read-parallel -Q -Z%d -z%d -f --gamma=0 --no-feature-limit --no-tile-size-limit -r1 -o tiles/centroids%s.mbtiles  ./tmp/centroids%s.geojson'%(start,end,year,year))
os.system('tippecanoe --read-parallel -Q -Z%d -z%d -f -ab -o tiles/outputarea%s.mbtiles --drop-densest-as-needed --generate-id ./tmp/OA%s.geojson'%(start,end,year,year))



print('join')
os.system('tile-join -o ./tiles/area_centroids_6_9.mbtiles -f ./tiles/centroids%s.mbtiles ./tiles/outputarea%s.mbtiles'%(year,year))

# --generate-id
# --coalesce-densest-as-needed

# tippecanoe -o tiles/test.mbtiles -Z9 -z10 -f ./data/tile_OA11.geojson ./data/tile_centroids11.geojson


# gdf_base = gdf_base.append(df,ignore_index=True)
# gdf_base .to_file("myfile.gpkg", layer="layer1", driver="GPKG")

save
process


For layer 0, using name "centroids01"
./tmp/centroids01.geojson:10944: Found ] at top level
./tmp/centroids01.geojson:10930: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","crs":{"type":"name","properties":{"name":"urn:ogc:def:crs:OGC:1.3:CRS84"}},"features":[]}
175434 features, 3073223 bytes of geometry, 16 bytes of separate metadata, 1929902 bytes of string pool

For layer 0, using name "OA01"
./tmp/OA01.geojson:23385: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","crs":{"type":"name","properties":{"name":"urn:ogc:def:crs:OGC:1.3:CRS84"}},"features":[]}
./tmp/OA01.geojson:9195: Found ] at top level
175434 features, 50496870 bytes of geometry, 895 bytes of separate metadata, 1929838 bytes of string pool
tile 6/32/21 size is 626726 with detail 12, >500000    
Going to try keeping the sparsest 71.80% of the features to make it fit
tile 6/32/21 size is 518544 with detail 12, >500000    
Going to try 

join



tile-join: invalid option -- P
Usage: tile-join [-f] [-i] [-pk] [-pC] [-c joins.csv] [-X] [-x exclude ...] -o new.mbtiles source.mbtiles ...


256

In [25]:
# base  = gpd.read_file('data/OA2011.geojson')[['OA01CD', 'LAD01CD',
#        'geometry']]
# centroids = gpd.read_file('data/centroids2011.geojson')[['lsoa01cd', 'geometry']]

# base.to_file("tmp/OA11.geojson", driver="GeoJSON")
# centroids.to_file("tmp/centroids11.geojson", driver="GeoJSON")

# os.system('tippecanoe -Z6 -z9 -f -o tiles/area_centroids_6_9.mbtiles --coalesce-densest-as-needed --generate-id ./tmp/OA%d.geojson ./tmp/centroids%d.geojson'%(11,11))


os.system('tippecanoe --read-parallel -Q -P -Z0 -z6 -f -o ./tiles/countries2.mbtiles '+ floc)

os.system('tile-join -o ./tiles/countries.mbtiles -f ./tiles/centroids%s.mbtiles ./tiles/countries2.mbtiles'%(year))


For layer 0, using name "county"
tmp/county.geojson:19: Found ] at top level
tmp/county.geojson:49: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","crs":{"type":"name","properties":{"name":"urn:ogc:def:crs:OGC:1.3:CRS84"}},"features":[]}
216 features, 1299508 bytes of geometry, 16 bytes of separate metadata, 2436 bytes of string pool

Tile 7/63/42 size is 533143, >500000. Skipping this tile
.Tile 6/31/20 size is 614249, >500000. Skipping this tile
.Tile 6/31/21 size is 673816, >500000. Skipping this tile


0

In [26]:
# merge
os.system('tile-join -o ./tiles/merged.mbtiles -f ./tiles/countries.mbtiles ./tiles/area_centroids_6_9.mbtiles --attribution=ONS')

Tile 8/128/85 size is 508237, >500000. Skipping this tile
.Tile 8/126/83 size is 574787, >500000. Skipping this tile
.Tile 7/64/42 size is 532895, >500000. Skipping this tile
.Tile 6/32/21 size is 511550, >500000. Skipping this tile
.Tile 9/252/165 size is 555523, >500000. Skipping this tile
.Tile 8/127/84 size is 536563, >500000. Skipping this tile


0

In [20]:
# tippecanoe -z3 -o countries-z3.mbtiles --coalesce-densest-as-needed ne_10m_admin_0_countries.geojson
# tippecanoe -zg -Z4 -o states-Z4.mbtiles --coalesce-densest-as-needed --extend-zooms-if-still-dropping ne_10m_admin_1_states_provinces.geojson
# tile-join -o states-countries.mbtiles countries-z3.mbtiles states-Z4.mbtiles

In [27]:
## convert to static server

os.system('rm -rf ./tiles/tileserver; mb-util --image_format=pbf ./tiles/merged.mbtiles ./tiles/tileserver')
os.system('cd ./tiles/tileserver && find . -type f | xargs -n1 -P 1 -t -I % gzip -d -r -S .pbf % 2>&1')
os.system('cd ./tiles/tileserver && find . -type f | xargs -n1 -P 1 -t -I % mv % %.pbf 2>&1')


DEBUG:mbutil.util:Exporting MBTiles to disk
DEBUG:mbutil.util:./tiles/merged.mbtiles --> ./tiles/tileserver
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:4 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:5 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:6 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:7 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:8 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:9 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:10 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:11 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:12 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:13 / 6461 tiles exported
DEBU

INFO:mbutil.util:550 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:551 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:552 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:553 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:554 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:555 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:556 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:557 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:558 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:559 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:560 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:561 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:562 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:563 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:564

DEBUG:mbutil.util:flipping
INFO:mbutil.util:1168 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1169 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1170 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1171 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1172 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1173 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1174 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1175 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1176 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1177 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1178 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1179 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1180 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1181 / 6461 tiles exported
DEBUG:

DEBUG:mbutil.util:flipping
INFO:mbutil.util:1836 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1837 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1838 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1839 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1840 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1841 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1842 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1843 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1844 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1845 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1846 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1847 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1848 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1849 / 6461 tiles exported
DEBUG:

DEBUG:mbutil.util:flipping
INFO:mbutil.util:2869 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2870 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2871 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2872 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2873 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2874 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2875 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2876 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2877 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2878 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2879 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2880 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2881 / 6461 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2882 / 6461 tiles exported
DEBUG:

...

mv ./12/1990/1357 ./12/1990/1357.pbf
mv ./12/1990/1385 ./12/1990/1385.pbf
mv ./12/1990/1382 ./12/1990/1382.pbf
mv ./12/1990/1379 ./12/1990/1379.pbf
mv ./12/1990/1383 ./12/1990/1383.pbf
mv ./12/1990/1384 ./12/1990/1384.pbf
mv ./12/1990/1352 ./12/1990/1352.pbf
mv ./12/1990/1355 ./12/1990/1355.pbf
mv ./12/1990/1354 ./12/1990/1354.pbf
mv ./12/1990/1353 ./12/1990/1353.pbf
mv ./12/1999/1375 ./12/1999/1375.pbf
mv ./12/1999/1381 ./12/1999/1381.pbf
mv ./12/1999/1372 ./12/1999/1372.pbf
mv ./12/1999/1328 ./12/1999/1328.pbf
mv ./12/1999/1326 ./12/1999/1326.pbf
mv ./12/1999/1373 ./12/1999/1373.pbf
mv ./12/1999/1380 ./12/1999/1380.pbf
mv ./12/1999/1374 ./12/1999/1374.pbf
mv ./12/1999/1327 ./12/1999/1327.pbf
mv ./12/1999/1329 ./12/1999/1329.pbf
mv ./12/1999/1334 ./12/1999/1334.pbf
mv ./12/1999/1333 ./12/1999/1333.pbf
mv ./12/1999/1356 ./12/1999/1356.pbf
mv ./12/1999/1369 ./12/1999/1369.pbf
mv ./12/1999/1351 ./12/1999/1351.pbf
mv ./12/1999/1367 ./12/1999/1367.pbf
mv ./12/1999/1358 ./12/1999/1358.pbf
m

mv ./12/2060/1366 ./12/2060/1366.pbf
mv ./12/2060/1350 ./12/2060/1350.pbf
mv ./12/2060/1357 ./12/2060/1357.pbf
mv ./12/2060/1368 ./12/2060/1368.pbf
mv ./12/2060/1349 ./12/2060/1349.pbf
mv ./12/2060/1340 ./12/2060/1340.pbf
mv ./12/2060/1347 ./12/2060/1347.pbf
mv ./12/2060/1346 ./12/2060/1346.pbf
mv ./12/2060/1341 ./12/2060/1341.pbf
mv ./12/2060/1348 ./12/2060/1348.pbf
mv ./12/2060/1370 ./12/2060/1370.pbf
mv ./12/2060/1352 ./12/2060/1352.pbf
mv ./12/2060/1355 ./12/2060/1355.pbf
mv ./12/2060/1364 ./12/2060/1364.pbf
mv ./12/2060/1337 ./12/2060/1337.pbf
mv ./12/2060/1339 ./12/2060/1339.pbf
mv ./12/2060/1365 ./12/2060/1365.pbf
mv ./12/2060/1354 ./12/2060/1354.pbf
mv ./12/2060/1353 ./12/2060/1353.pbf
mv ./12/2060/1338 ./12/2060/1338.pbf
mv ./12/2060/1336 ./12/2060/1336.pbf
mv ./12/2034/1375 ./12/2034/1375.pbf
mv ./12/2034/1372 ./12/2034/1372.pbf
mv ./12/2034/1344 ./12/2034/1344.pbf
mv ./12/2034/1343 ./12/2034/1343.pbf
mv ./12/2034/1317 ./12/2034/1317.pbf
mv ./12/2034/1328 ./12/2034/1328.pbf
m

mv ./12/2002/1379 ./12/2002/1379.pbf
mv ./12/2002/1346 ./12/2002/1346.pbf
mv ./12/2002/1341 ./12/2002/1341.pbf
mv ./12/2002/1348 ./12/2002/1348.pbf
mv ./12/2002/1383 ./12/2002/1383.pbf
mv ./12/2002/1377 ./12/2002/1377.pbf
mv ./12/2002/1370 ./12/2002/1370.pbf
mv ./12/2002/1352 ./12/2002/1352.pbf
mv ./12/2002/1355 ./12/2002/1355.pbf
mv ./12/2002/1330 ./12/2002/1330.pbf
mv ./12/2002/1337 ./12/2002/1337.pbf
mv ./12/2002/1339 ./12/2002/1339.pbf
mv ./12/2002/1354 ./12/2002/1354.pbf
mv ./12/2002/1353 ./12/2002/1353.pbf
mv ./12/2002/1338 ./12/2002/1338.pbf
mv ./12/2002/1336 ./12/2002/1336.pbf
mv ./12/2002/1331 ./12/2002/1331.pbf
mv ./12/2061/1344 ./12/2061/1344.pbf
mv ./12/2061/1343 ./12/2061/1343.pbf
mv ./12/2061/1342 ./12/2061/1342.pbf
mv ./12/2061/1345 ./12/2061/1345.pbf
mv ./12/2061/1356 ./12/2061/1356.pbf
mv ./12/2061/1369 ./12/2061/1369.pbf
mv ./12/2061/1351 ./12/2061/1351.pbf
mv ./12/2061/1367 ./12/2061/1367.pbf
mv ./12/2061/1335 ./12/2061/1335.pbf
mv ./12/2061/1366 ./12/2061/1366.pbf
m

mv ./12/2004/1346 ./12/2004/1346.pbf
mv ./12/2004/1341 ./12/2004/1341.pbf
mv ./12/2004/1348 ./12/2004/1348.pbf
mv ./12/2004/1383 ./12/2004/1383.pbf
mv ./12/2004/1377 ./12/2004/1377.pbf
mv ./12/2004/1370 ./12/2004/1370.pbf
mv ./12/2004/1384 ./12/2004/1384.pbf
mv ./12/2004/1352 ./12/2004/1352.pbf
mv ./12/2004/1355 ./12/2004/1355.pbf
mv ./12/2004/1330 ./12/2004/1330.pbf
mv ./12/2004/1337 ./12/2004/1337.pbf
mv ./12/2004/1339 ./12/2004/1339.pbf
mv ./12/2004/1354 ./12/2004/1354.pbf
mv ./12/2004/1353 ./12/2004/1353.pbf
mv ./12/2004/1338 ./12/2004/1338.pbf
mv ./12/2004/1336 ./12/2004/1336.pbf
mv ./12/2004/1331 ./12/2004/1331.pbf
mv ./12/2032/1297 ./12/2032/1297.pbf
mv ./12/2032/1299 ./12/2032/1299.pbf
mv ./12/2032/1298 ./12/2032/1298.pbf
mv ./12/2032/1296 ./12/2032/1296.pbf
mv ./12/2032/1375 ./12/2032/1375.pbf
mv ./12/2032/1372 ./12/2032/1372.pbf
mv ./12/2032/1344 ./12/2032/1344.pbf
mv ./12/2032/1343 ./12/2032/1343.pbf
mv ./12/2032/1317 ./12/2032/1317.pbf
mv ./12/2032/1328 ./12/2032/1328.pbf
m

0

In [28]:
os.system("git add -A; git commit -m 'recompute tiles'; git push")

[main 133f0a09] recompute tiles
 5971 files changed, 211030 insertions(+), 175696 deletions(-)
 rewrite tiles/tileserver/0/0/0.pbf (97%)
 rewrite tiles/tileserver/1/0/0.pbf (91%)
 rewrite tiles/tileserver/1/1/0.pbf (95%)
 create mode 100644 tiles/tileserver/10/511/340.pbf
 rewrite tiles/tileserver/2/1/1.pbf (78%)
 rewrite tiles/tileserver/2/2/1.pbf (87%)
 rewrite tiles/tileserver/3/4/2.pbf (73%)
 rewrite tiles/tileserver/4/8/5.pbf (61%)
 delete mode 100644 tiles/tileserver/6/31/20.pbf
 delete mode 100644 tiles/tileserver/6/31/21.pbf
 delete mode 100644 tiles/tileserver/6/32/21.pbf
 create mode 100644 tiles/tileserver/7/63/41.pbf
 delete mode 100644 tiles/tileserver/7/64/42.pbf
 create mode 100644 tiles/tileserver/8/126/82.pbf
 delete mode 100644 tiles/tileserver/8/126/83.pbf
 delete mode 100644 tiles/tileserver/8/127/84.pbf
 create mode 100644 tiles/tileserver/8/127/85.pbf
 delete mode 100644 tiles/tileserver/8/128/85.pbf
 delete mode 100644 tiles/tileserver/9/252/165.pbf
 create mode 

remote: warning: See http://git.io/iEPt8g for more information.        
remote: warning: File tiles/merged.mbtiles is 83.89 MB; this is larger than GitHub's recommended maximum file size of 50.00 MB        
remote: warning: GH001: Large files detected. You may want to try Git Large File Storage - https://git-lfs.github.com.        
To https://github.com/wolfiex/ONStileBuilder.git
   9434ea0d..133f0a09  main -> main


0